In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-11 01:03:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.63MB/s    in 0.2s    

2021-10-11 01:03:17 (5.63 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://lscherger-bucket.s3.amazonaws.com/amazon_reviews_us_Health_Personal_Care_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [ ]:
# Create the vine_table. DataFrame
vine_table_df = df.select(['review_id', 'star_rating', 'helpful_votes','total_votes','vine','verified_purchase'])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [ ]:
high_votes_df = vine_table_df.filter("total_votes>=20")
high_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [ ]:
helpful_percent_df = vine_table_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R37R181DDM2UUL|          4|            2|          2|   N|                Y|
|R2H4PFXM303UPQ|          4|            3|          3|   N|                Y|
| RZ9HASH6YNE4J|          5|            1|          1|   N|                Y|
| RR3HYBU0LOG05|          5|            2|          3|   N|                Y|
| R45LXAPWF1BYU|          5|            9|         10|   N|                Y|
| RMHV44UUQNFKD|          5|            2|          3|   N|                Y|
|R2D1QHB72QYC2W|          5|            2|          2|   N|                Y|
| RLBGTWG2E2JQ1|          5|           13|         16|   N|     

In [ ]:
vine_review_df = vine_table_df.filter("vine=='Y'")
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BIFVKMZQPGDC|          5|            8|          9|   Y|                N|
|R35E49JZWFJDBK|          5|            1|          1|   Y|                N|
|R2PIHINVNVGDL9|          4|            0|          0|   Y|                N|
|R2KQ11RRU9Z5N4|          5|            0|          0|   Y|                N|
|R2H9CLVRH00TAJ|          5|            2|          2|   Y|                N|
|R3PCFK0001XURF|          5|            0|          0|   Y|                N|
|R20DLO2PLAX2WM|          5|            1|          1|   Y|                N|
|R34TE2UEOXH0LB|          4|            0|          1|   Y|                N|
|R2ERMN4JIX94R6|          2|            0|          0|   Y|                N|
|R2NBCQETHBBFL6|          3|            1|          2|   Y|     

In [ ]:
no_vine_review_df = vine_table_df.filter("vine=='N'")
no_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [49]:
clean_vines_df=vine_table_df.drop_duplicates(["review_id"])
clean_vines_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1006MZQZESJLU|          5|            0|          0|   N|                Y|
| R100KFB0HIL17|          3|            1|          2|   N|                Y|
|R100VCRMA6O31M|          2|            0|          0|   N|                Y|
|R10123RVTQX469|          1|           10|         12|   N|                Y|
|R101AA8NMPCECG|          5|            3|          5|   N|                Y|
|R101H9BZ4XSO4P|          4|            4|          4|   N|                Y|
|R101OASBXZOKB5|          4|            0|          0|   N|                Y|
|R101RBWHDT9BUD|          5|            0|          0|   N|                Y|
|R101WUH16PCM6G|          5|            0|          0|   N|                Y|
|R1020VRZ9DVRFC|          4|            3|          3|   N|     

In [52]:
# Calculate total number of reviews
clean_vines_df.count()

5331449

In [53]:
# Calculate total number of 5 star reviews
clean_vines_df.filter("star_rating=5").count()

3358086

In [60]:
# Calculate percentage of 5 star reviews for vine reviews
vine_five_star = vine_review_df.filter("star_rating=5").count()
vine_total = vine_review_df.count()
vine_five_star_percentage = ((vine_five_star)/(vine_total))*100
vine_five_star_percentage

40.98857178542434

In [62]:
# Calculate percentage of 5 star reviews for non - vine reviews
no_vine_five_star = no_vine_review_df.filter("star_rating=5").count()
no_vine_total = no_vine_review_df.count()
no_vine_five_star_percentage = ((no_vine_five_star)/(no_vine_total))*100
no_vine_five_star_percentage

63.12209283345055